In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn

In [31]:
data  = pd.read_csv("./credit-a.csv",header=None)
data.head()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 653 entries, 0 to 652
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       653 non-null    int64  
 1   1       653 non-null    float64
 2   2       653 non-null    float64
 3   3       653 non-null    int64  
 4   4       653 non-null    int64  
 5   5       653 non-null    int64  
 6   6       653 non-null    int64  
 7   7       653 non-null    float64
 8   8       653 non-null    int64  
 9   9       653 non-null    int64  
 10  10      653 non-null    int64  
 11  11      653 non-null    int64  
 12  12      653 non-null    int64  
 13  13      653 non-null    int64  
 14  14      653 non-null    float64
 15  15      653 non-null    int64  
dtypes: float64(4), int64(12)
memory usage: 81.8 KB


In [32]:
X = data.iloc[:,:-1]
Y = data.iloc[:,-1].replace(-1,0)

In [33]:
# values 获取ndarray
X = torch.from_numpy(X.values).type(torch.float32)
Y = torch.from_numpy(Y.values.reshape(-1,1)).type(torch.float32)

In [39]:
#可以将多个层顺序连接
#Linear参数 分别是输入的形状和输出的形状
model = nn.Sequential(
    nn.Linear(15,1),
    nn.Sigmoid()
)

In [40]:
model

Sequential(
  (0): Linear(in_features=15, out_features=1, bias=True)
  (1): Sigmoid()
)

In [46]:
#二元分类交叉熵损失
loss_fn = nn.BCELoss()

In [43]:
#Adam 会考虑之前下降了多少 进行加权 随机梯度下降不会考虑之前
opt = torch.optim.Adam(model.parameters(),lr=0.0001)

In [44]:
#一个值一个值训练对于异常值十分敏感，使用小批次训练
batches  = 16
no_of_batch = 653 // 16

In [45]:
epoches = 1000

In [48]:
for epoch in range(epoches):
    for batch in range(no_of_batch):
        start = batch*batches
        end = start + batches
        x = X[start:end]
        y = Y[start:end]
        y_pred = model(x)
        loss = loss_fn(y_pred,y)
        opt.zero_grad()
        loss.backward()
        opt.step()

In [49]:
model.state_dict()

OrderedDict([('0.weight',
              tensor([[-2.9938e-01, -8.7434e-03,  3.6001e-02,  1.8289e-01,  3.6037e-01,
                       -5.2897e-03,  2.0719e-01, -1.5503e-01,  4.4813e+00,  3.1905e-01,
                       -1.4352e-01, -2.8522e-01, -2.2652e-01,  2.1602e-03,  9.8745e-05]])),
             ('0.bias', tensor([-0.5046]))])

In [54]:
((model(X).data.numpy() > 0.5).astype(np.int) == Y.numpy()).mean()

C:\Users\warr\AppData\Local\Temp\ipykernel_22700\3049843564.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  ((model(X).data.numpy() > 0.5).astype(np.int) == Y.numpy()).mean()


0.8499234303215927